In [1]:
from fastai.imports import *
from fastai.structured import *

In [2]:
!ls

 Data_Train.xlsx	  sample.csv		   Test_set.xlsx
'duration column.ipynb'   Sample_submission.xlsx   ticket.ipynb
 edit.ipynb		  sub.csv		   train.csv
 fastai			  submission.xlsx	   trainRaw
 featherwali.ipynb	  test.csv		   tsp.csv
 final.csv		  test-raw		   tst.csv
'lgbm model.ipynb'	  testRaw		   Untitled.ipynb


In [21]:
data=pd.read_excel('Data_Train.xlsx')

In [22]:
data.head(15)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625


In [23]:
data.shape

(10683, 11)

In [24]:
import re
corpus=[]
for i in range(0,10683):
    #dur=re.findall('[0-9][0-9]h', str(data['Duration'][i]))
    #dur=re.sub(r'\D', "", str(dur))
    dur=str(data['Duration'][i])
    
    corpus.append(dur.split(" "))
data['hour']=corpus

In [25]:
data['hour'].head()

0    [2h, 50m]
1    [7h, 25m]
2        [19h]
3    [5h, 25m]
4    [4h, 45m]
Name: hour, dtype: object

In [26]:
corpus2=[]

for k in range(0,10683):
    dus=data['hour'][k]
    kk=dus[0]
    ur=re.sub(r'\D', "", str(kk))
    #
    corpus2.append(ur)
    #mnt.append(us)
data['hourstring']=corpus2

In [27]:
data['hourstring'].head()

0     2
1     7
2    19
3     5
4     4
Name: hourstring, dtype: object

In [28]:
data['hourstring']=data['hourstring'].astype(int)

In [29]:
mnt=[]
for j in range(0,10683):
    lamba=len(data['hour'][j])
    du=data['hour'][j]
    if lamba==2:
        kks=du[1]
        us=re.sub(r'\D', "", str(kks))
        mnt.append(us)
    elif lamba==1:
        mnt.append(0)
data['min']=mnt

In [30]:
data['min']=data['min'].astype(int)

In [31]:
print(data['min'][3])

25


In [32]:
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,hour,hourstring,min
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,"[2h, 50m]",2,50
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,"[7h, 25m]",7,25
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,[19h],19,0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,"[5h, 25m]",5,25
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,"[4h, 45m]",4,45


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
Airline            10683 non-null object
Date_of_Journey    10683 non-null object
Source             10683 non-null object
Destination        10683 non-null object
Route              10682 non-null object
Dep_Time           10683 non-null object
Arrival_Time       10683 non-null object
Duration           10683 non-null object
Total_Stops        10682 non-null object
Additional_Info    10683 non-null object
Price              10683 non-null int64
hour               10683 non-null object
hourstring         10683 non-null int64
min                10683 non-null int64
dtypes: int64(3), object(11)
memory usage: 1.1+ MB


In [34]:
data['kitnaTimelaga']=(data['hourstring']*3600)+(data['min']*60)

In [37]:
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Total_Stops,Additional_Info,Price,kitnaTimelaga
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,non-stop,No info,3897,10200
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,2 stops,No info,7662,26700
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,2 stops,No info,13882,68400
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,1 stop,No info,6218,19500
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,1 stop,No info,13302,17100


In [36]:
data.drop(['Duration', 'hourstring','min','hour'], axis=1,inplace=True)